<a href="https://colab.research.google.com/github/himanshu804/codsoft_taskno.-1/blob/main/Fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:     
  
      1. id: unique id for a news article
      2. title: the title of a news article
      3. author: author of the news article
      4. text: the text of the article; could be incomplete
      5. label: a label that marks wther the news article is real or fake
             1: fake
             0: real


Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# printing the  stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [ ]:
 # fixing some error in Dataset
import csv

def add_missing_quote(filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        data = [row for row in reader]

    # Check the last row and add the missing close quote character if needed
    last_row = data[-1]
    for i, cell in enumerate(last_row):
        if cell.count('"') % 2 != 0:
            last_row[i] = cell + '"'
            break

    # Overwrite the file with the corrected data
    with open(filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(data)

# Replace '/content/train.csv' with the path to your actual train.csv file
add_missing_quote('/content/train.csv')


In [ ]:
# loading the dataset to a pandas Dataframe
dataset = pd.read_csv('/content/train.csv')


In [ ]:
dataset.shape

(3776, 5)

In [ ]:
import numpy as np

def func(value):
    value_str = str(value).replace("''", ' ')

    # Check if the value is not an empty string or NaN
    if value_str.strip() != '' and value_str != 'nan':
        value_float = float(value_str)
        return int(value_float)
    else:
        return np.nan  # or handle the case in a way that makes sense for your data

dataset['label'] = dataset['label'].apply(func)

# Convert the column to integers, handling NaN values
dataset['label'] = dataset['label'].astype('Int64')


In [ ]:
dataset.label

0          1
1          0
2          1
3          1
4          1
        ... 
3771       0
3772       1
3773       1
3774       1
3775    <NA>
Name: label, Length: 3776, dtype: Int64

In [ ]:
# print the first 5 rows of the dataframe
dataset.head()

dataset.dtypes

id         int64
title     object
author    object
text      object
label      Int64
dtype: object

In [ ]:
# counting the number of missing values in the dataset
dataset.isnull().sum()

id          0
title     100
author    389
text        9
label       1
dtype: int64

In [ ]:
# replacing the null balue with empty string
dataset.dropna(inplace=True)



In [ ]:
# merging the author name and news title
dataset['content'] = dataset['author']+''+dataset['title']





In [ ]:
print(dataset['content'])

0       Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1       Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2       Consortiumnews.comWhy the Truth Might Get You ...
3       Jessica Purkiss15 Civilians Killed In Single U...
4       Howard PortnoyIranian woman jailed for fiction...
                              ...                        
3770    Lucas Nolan’Deport Fat People’ Posters Appear ...
3771    Dan RiehlRep. Jim Jordan: Working with Dems on...
3772    Colin TaylorCNBC Just Confirmed Comey Is Abusi...
3773    Jenn RyanCan You Get a Stunning and Healthy Sm...
3774    Sean Adl-TabatabaiProof of Past Life? Teen Wak...
Name: content, Length: 3286, dtype: object


In [ ]:
#separating the data & label
x = dataset.drop(columns = 'label', axis=1)
y = dataset['label']

In [ ]:
print(x)
print(y)

        id                                              title  \
0        0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
3770  3770  ’Deport Fat People’ Posters Appear At CU Bould...   
3771  3771  Rep. Jim Jordan: Working with Dems on Health C...   
3772  3772  CNBC Just Confirmed Comey Is Abusing His Power...   
3773  3773  Can You Get a Stunning and Healthy Smile with ...   
3774  3774  Proof of Past Life? Teen Wakes From Coma Speak...   

                  author                                               text  \
0          Darrell Lucus  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        Daniel J. Flynn  Ever get the feeling your life circ

**Stemming**

its the process of reducing a word to its root word

example:
actor, actress , acting ---> act (root word)

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-z A-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
dataset['content'] = dataset['content'].apply(stemming)

In [ ]:
print(dataset['content'])

0       darrel lucushous dem aid even see comey letter...
1       daniel j flynnflynn hillari clinton big woman ...
2               consortiumnew comwhi truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoyiranian woman jail fiction unpub...
                              ...                        
3770    luca nolan deport fat peopl poster appear cu b...
3771    dan riehlrep jim jordan work dem health care w...
3772    colin taylorcnbc confirm comey abus power help...
3773      jenn ryancan get stun healthi smile coconut oil
3774    sean adl tabatabaiproof past life teen wake co...
Name: content, Length: 3286, dtype: object


In [ ]:
#separating th data and label
X = dataset['content'].values
Y = dataset['label'].values

In [ ]:
print(X)

['darrel lucushous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynnflynn hillari clinton big woman campu breitbart'
 'consortiumnew comwhi truth might get fire' ...
 'colin taylorcnbc confirm comey abus power help trump'
 'jenn ryancan get stun healthi smile coconut oil'
 'sean adl tabatabaiproof past life teen wake coma speak fluent spanish']


In [ ]:
print(Y)

<IntegerArray>
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
 ...
 0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
Length: 3286, dtype: Int64


In [ ]:
Y.shape

(3286,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 8432)	0.29053434517272697
  (0, 7258)	0.2564119758484349
  (0, 4861)	0.37377956673474605
  (0, 4709)	0.31212648038529567
  (0, 4182)	0.244016670817193
  (0, 2621)	0.23733312517115746
  (0, 2037)	0.2940941086012197
  (0, 1936)	0.3557471949506701
  (0, 1603)	0.2548513490211767
  (0, 1345)	0.37377956673474605
  (0, 200)	0.2812999372105705
  (1, 8875)	0.3760793942932467
  (1, 3691)	0.24800536983924382
  (1, 2916)	0.4969767257199752
  (1, 1916)	0.33012465675962566
  (1, 1520)	0.24506380563784005
  (1, 1235)	0.4652396721012951
  (1, 1064)	0.18153226466565983
  (1, 852)	0.36281026221105117
  (2, 8406)	0.37824986518701337
  (2, 5249)	0.4491063917374364
  (2, 3145)	0.3107169125063554
  (2, 2840)	0.34247848287309907
  (2, 1688)	0.4177053569650337
  (2, 1641)	0.5166393444180944
  :	:
  (3283, 6323)	0.3275648437798642
  (3283, 3643)	0.33400354385503156
  (3283, 1664)	0.3711559769167871
  (3283, 1603)	0.3255711518659902
  (3283, 1573)	0.36694557771746644
  (3283, 29)	0.38606204244443215
  (32

In [ ]:
print(Y)

<IntegerArray>
[1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
 ...
 0, 1, 1, 0, 1, 0, 0, 1, 1, 1]
Length: 3286, dtype: Int64


Splitting the dataset to training & test data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

In [ ]:
X_train.shape

(2628, 9026)

In [ ]:
Y_train.shape

(2628,)

Training the Model: Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Assuming you have X_train and Y_train
# Encode categorical labels
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)

# Create a logistic regression model
model = LogisticRegression()

# Fit the model on the training data
model.fit(X_train, Y_train_encoded)


LogisticRegression()

Evalution

accuracy score

In [ ]:
#accuracy score on the train data
from sklearn.metrics import accuracy_score

# Ensure both Y_train and X_train_prediction are binary labels
# (if not, convert them to binary labels)
Y_train_binary = (Y_train > 0.5).astype(int)
X_train_prediction_binary = (X_train_prediction > 0.5).astype(int)

# Check the unique values in Y_train and X_train_prediction
print("Unique values in Y_train:", np.unique(Y_train_binary))
print("Unique values in X_train_prediction:", np.unique(X_train_prediction_binary))

# Calculate accuracy score
training_data_accuracy = accuracy_score(Y_train_binary, X_train_prediction_binary)


Unique values in Y_train: [0 1]
Unique values in X_train_prediction: [0 1]


In [ ]:
print(training_data_accuracy)

0.9794520547945206


In [ ]:
# accuracy score on the test data
from sklearn.metrics import accuracy_score

# Ensure both Y_test and X_test_prediction are binary labels
# (if not, convert them to binary labels)
Y_test_binary = (Y_test > 0.5).astype(int)
X_test_prediction_binary = (X_test_prediction > 0.5).astype(int)

# Check the unique values in Y_test and X_test_prediction
print("Unique values in Y_test:", np.unique(Y_test_binary))
print("Unique values in X_test_prediction:", np.unique(X_test_prediction_binary))

# Calculate accuracy score
test_data_accuracy = accuracy_score(Y_test_binary, X_test_prediction_binary)


Unique values in Y_test: [0 1]
Unique values in X_test_prediction: [0 1]


In [ ]:
print(test_data_accuracy)

0.9376899696048632


Making a Predictive System

In [ ]:
X_new = X_test[33]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [ ]:
print(Y_test[600])

1
